In this part 2 notebook, we will increase all the network properties to achieve a higher model aacuracy. Later, the model will be saved, load and predict new concrete strength with new user defined parameters.

<h2><center> Building a Concrete Compressive Strength Model using Deep Learning Keras Library </center></h2>

<img src = "tf-keras.png" width = 500>

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'

In [3]:
df = pd.read_csv(url)

In [4]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


### MODEL - Construct Model to Predict and Forecast.

#### Split Data to Predictors and Target

In [5]:
X = df.iloc[:,:-1]
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
y = df.iloc[:,-1]
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalizing Data

In [7]:
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Save number of feature columns, <strong><i>n_cols</i></strong> to use later in model development.

In [8]:
n_cols = X.shape[1]
n_cols

8

#### Importing Libraries

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import keras

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Building the Model

<strong>Network Properties:</strong>
<ul>
  <li>Hidden Layers: 7</li>
  <li>Nodes: 25</li>
  <li>Activation Function: ReLU</li>
  <li>Optimizer: Adam</li>
  <li>Loss Function: Mean Squared Error</li>
  <li>Epochs: 250</li>
</ul>

In [10]:
mse = []
r2 = []

for i in range(50):
    
    #Split Data to Train and Test Set
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = 0.3)

    #Create model
    model = Sequential()
    model.add(Dense(25, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))

    #Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')

    #fit the model
    model.fit(X_train, y_train, epochs=250, verbose=0)

    #predict output on test set
    y_pred = model.predict(X_test)
    
    mse.append(mean_squared_error(y_test, y_pred))
    r2.append(r2_score(y_test, y_pred))

In [11]:
print('mse_Mean: {:.2f}'.format(np.mean(mse)))
print('mse_StdDev: {:.2f}'.format(np.std(mse)))

mse_Mean: 32.41
mse_StdDev: 5.47


In [12]:
print('R^2_Mean: {:.2f}'.format(np.mean(r2)))
print('R^2_StdDev: {:.2f}'.format(np.std(r2)))

R^2_Mean: 0.88
R^2_StdDev: 0.02


In [13]:
from IPython.display import HTML, display
import tabulate

tabletest = [['PART','MSE: Mean','MSE: StdDev','R^2: Mean','R^2: StdDev'],
         ['2', round(np.mean(mse),2), round(np.std(mse),2), round(np.mean(r2),2), round(np.std(r2),2)]]

display(HTML(tabulate.tabulate(tabletest, tablefmt='html')))

PART,MSE: Mean,MSE: StdDev,R^2: Mean,R^2: StdDev
2,32.41,5.47,0.88,0.02


#### Comparing the results from Part 1:
- mean squared error has gone down from 95.28 to 32.41
- R^2 has gone up from 0.65 to 0.88

<b>which means the overall accuracy has gone up compared from the previous run.</b>

### SAVING MODEL

In [14]:
model.save('keras_reg.h5')

# Predict New Concrete Strength Value with Pretrained Model

## Import Module

In [15]:
import numpy as np
from keras.models import load_model

## Load Pretrained Model

In [16]:
keras_reg = load_model('keras_reg.h5')

## Insert New Parameters

In [17]:
#Cement
x1 = 1540.0

#Blast Furnace Slag
x2 = 250.0

#Fly Ash
x3 = 0.0

#Water
x4 = 162.0

#Superplasticizer
x5 = 2.5

#Coarse Aggregate
x6 = 1055.0

#Fine Aggregate
x7 = 676.0

#Age
x8 = 450

In [18]:
X = np.array([[x1, x2, x3, x4, x5, x6, x7, x8]])
X_norm = (X - X.mean()) / X.std()
strength_pred = keras_reg.predict(X_norm)

## New Prediction

In [19]:
print('The predicted concrete strength is {:.2f}'.format(strength_pred[0][0]))

The predicted concrete strength is 71.87
